In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
import imageio as iio
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops
from skimage.registration import phase_cross_correlation as pcc
from skimage.io import imread as imread

In [131]:
dirname = r'/data/old_trap/20210405/bead1/Height_pumpingdown/run2_fb1/'
dirname = r'/data/old_trap/20210329/bead2/height_long/run1/'
file_prefix = 'image_'
## Load all filenames in directory
## Load all filenames in directory
files = []
[files.append(file_) for file_ in os.listdir(dirname) if file_.startswith(file_prefix) if file_.endswith('.bmp')]
files.sort(key=lambda ff: int(os.path.splitext(ff)[0].split('_')[-1])) ## sort by index
zeroth_image=imread(os.path.join(dirname+files[0]), as_gray=True)
len(files)

2799

In [115]:
# Y1 = []
height_run1 = []
for ff_ in files:
    image = imread(os.path.join(dirname+ff_), as_gray=True)
    shift,_,_ = pcc(zeroth_image,image,upsample_factor=100)    
    height_run1.append(shift[0])

In [136]:
def get_height(i):
    image = imread(os.path.join(dirname+files[i]), as_gray=True)
    shift,_,_ = pcc(zeroth_image,image,upsample_factor=1000)    
    return shift[0]

In [137]:
height_run1_3 = np.array(Parallel(n_jobs=40)(delayed(get_height)(i) for i in tqdm(range(0, 2799))))


100%|██████████| 2799/2799 [01:48<00:00, 25.85it/s]

In [130]:
_, ax = plt.subplots()
ax.plot(range(len(height_run1)), height_run1-np.mean(height_run1), '.-')
ax.plot(range(len(height_run2)), height_run2-np.mean(height_run2), '.-')
ax.plot(range(len(height_run3)), height_run3-np.mean(height_run3), '.-')
ax.plot(range(len(height_run4)), height_run4-np.mean(height_run4), '.-')
ax.set(ylim=(-0.2,0.2))
# ax.plot(range(len(Y2)), Y2-np.mean(Y2), '.-')
# ax.plot(range(len(Y1)), Y1-np.mean(Y1), '.-')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(-0.2, 0.2)]

In [144]:
height_run1_4 = [np.mean(height_run1_3[i:i+10]) for i in range(len(height_run1_3))]
height_run1_5 = [np.mean(height_run1_3[i:i+20]) for i in range(len(height_run1_3))]

In [145]:
_, ax = plt.subplots()
# ax.plot(range(len(height_run1)), height_run1-np.mean(height_run1), '.-')
# ax.plot(range(len(height_run1_2)), height_run1_2-np.mean(height_run1_2), '.-')
ax.plot(range(len(height_run1_3)), height_run1_3-np.mean(height_run1_3), '.-')
ax.plot(range(len(height_run1_4)), height_run1_4-np.mean(height_run1_4), '.-')
ax.plot(range(len(height_run1_5)), height_run1_5-np.mean(height_run1_5), '.-')
ax.set(ylim=(-0.2,0.2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(-0.2, 0.2)]

### Heigth log

In [31]:
filename = r'/data/old_trap/20210405/bead1/neutrality/height/w_height_fb_50V_41_Hz_x/height_log.txt'
filename = r'/data/old_trap/20210405/bead1/Height_pumpingdown/test13/height_log.txt'
filename = r'/data/old_trap/20210405/bead1/neutrality/height/w_height_fb_50V_41_Hz_x_smaller_counts_per_update/height_log.txt'
filename = r'/data/old_trap/20210429/bead1/height/overnight_20210504_1/height_log.txt'
# filename = r'/data/old_trap/20210405/bead1/neutrality/height/w_height_fb_50V_41_Hz_x/height_log.txt'
heights_log = np.genfromtxt(filename,delimiter='\t')

In [33]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

timestamp = 3640111724.4817362
times = heights_log[1]
date = datetime.fromtimestamp(times[-1])
date = date - relativedelta(years=66)
print(date.strftime("%m/%d/%Y, %H:%M:%S"))

05/06/2021, 01:23:30


In [37]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.plot(times-times[0], heights_log[0], '.-')
# ax.plot(range(len(heights_log2)), heights_log2, '.-')
# ax.set(ylim=(-0.4,0.1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [183]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.plot(range(len(heights_log)), heights_log, '.-', label='neutrality run')
ax.set(ylim=(-0.3,0.3))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [169]:
_,ax = plt.subplots()
_ = ax.hist(heights_log[:1000], bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …